In [28]:
import pandas as pd
import os
from etl.extract.extract import extract_data
from config.env_config import setup_env


In [ ]:
# You need to make sure that the notebook thinks its executing from the root of the project!

print(os.getcwd())
os.chdir('../')
print(os.getcwd())

setup_env(['run_etl', 'dev'])

In [29]:
# Extract the data
# extracted_data = extract_data()
# # Put the tuples into separate dataframes
# transactions = pd.DataFrame(extracted_data[0])
# customers = pd.DataFrame(extracted_data[1])

transactions = pd.read_csv('../tests/test_data/test_transactions.csv')


### Transform and clean the transactions data

#### Remove missing values

In [30]:
# remove rows with null values in transaction from the transaction dataframe
transactions = transactions.dropna(subset=['transaction_date'])

# remove rows with null values in amount from the transaction dataframe
transactions = transactions.dropna(subset=['amount'])

# See information about the transactions dataframe
transactions.info()

<class 'pandas.core.frame.DataFrame'>
Index: 10246 entries, 0 to 10499
Data columns (total 4 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   transaction_id    10246 non-null  int64 
 1   customer_id       10246 non-null  int64 
 2   transaction_date  10246 non-null  object
 3   amount            10246 non-null  object
dtypes: int64(2), object(2)
memory usage: 400.2+ KB


#### Remove invalid values

In [31]:
# Remove rows with the value of "INVALID" in the amount column

transactions = transactions[transactions['amount'] != 'INVALID']

# transactions.info()


#### Standardise the date format

In [32]:
# Convert all dates into dd/mm/yyyy format - write function to handle the different types of date formats

def standardise_date(date_str):
    if pd.isna(date_str) or date_str == '':
        return pd.NaT

    formats = [
        '%Y/%m/%d', '%Y-%m-%d', '%d %b %Y', '%b %d, %Y', '%d %B %Y',
        '%d-%m-%Y', '%d/%m/%Y'
    ]
    for fmt in formats:
        try:
            return pd.to_datetime(date_str, format=fmt)
        except ValueError:
            continue

    return pd.NaT


# Apply the parse_date function to the transaction_date column
transactions['transaction_date'] = (
    transactions['transaction_date'].apply(standardise_date)
)
transactions['transaction_date'] = (
    transactions['transaction_date'].dt.strftime('%d/%m/%Y')
)

# Display the DataFrame info
transactions.info()

<class 'pandas.core.frame.DataFrame'>
Index: 10205 entries, 0 to 10499
Data columns (total 4 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   transaction_id    10205 non-null  int64 
 1   customer_id       10205 non-null  int64 
 2   transaction_date  8977 non-null   object
 3   amount            10205 non-null  object
dtypes: int64(2), object(2)
memory usage: 398.6+ KB


#### Remove any duplicates after cleaning and transforming

In [33]:

# Drop duplicates

transactions.drop_duplicates(inplace=True)

transactions.info()

# how many nan in transaction_date?

nan_count = transactions['transaction_date'].isna().sum()

print(nan_count)

<class 'pandas.core.frame.DataFrame'>
Index: 9702 entries, 0 to 9981
Data columns (total 4 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   transaction_id    9702 non-null   int64 
 1   customer_id       9702 non-null   int64 
 2   transaction_date  8534 non-null   object
 3   amount            9702 non-null   object
dtypes: int64(2), object(2)
memory usage: 379.0+ KB
1168


In [34]:
transactions = transactions.dropna(subset=['transaction_date'])

transactions.info()

<class 'pandas.core.frame.DataFrame'>
Index: 8534 entries, 0 to 9981
Data columns (total 4 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   transaction_id    8534 non-null   int64 
 1   customer_id       8534 non-null   int64 
 2   transaction_date  8534 non-null   object
 3   amount            8534 non-null   object
dtypes: int64(2), object(2)
memory usage: 333.4+ KB


In [38]:
# export to results file

transactions.to_csv('../tests/test_data/expected_transactions_clean_results.csv', index=False)

---

### Clean and Standardise the customer data

In [7]:
# Do the similar for customers

customers.info()

NameError: name 'customers' is not defined

#### Remove missing values

In [ ]:
customers.dropna(inplace=True)

customers.info()

#### Standardise the country names to uppercase

In [ ]:
# Capitalize country names
customers['country'] = customers['country'].str.upper()

customers

#### Standardise the is_active column to True or False

In [ ]:
# convert all strings `active` or `inactive` to boolean values as well as integer values 0 and 1 to their boolean counterparts

def parse_active(active_str):
    if str(active_str.lower()) == 'active' or str(active_str) == '1':
        return True
    else:
        return False


# Apply the parse_active function to the active column
customers['is_active'] = customers['is_active'].apply(parse_active)

customers

#### Remove duplicates from the customers dataframe

In [ ]:
# Check for duplicates in the customers dataframe
duplicate_count = customers.duplicated().sum()
print(f"Number of duplicate rows in customers dataframe: {duplicate_count}")

In [ ]:
customers.drop_duplicates(inplace=True)

customers.info()

---

### Merge the dataframes

In [ ]:
# Merge the dataframes on customer_id

merged_data = pd.merge(transactions, customers, on='customer_id')

merged_data.info()

In [ ]:
merged_data.head()